In [36]:
binarize = True
model_type = "classification"
remove_trend = False # Bad parameters (decrease performance)
lag_features = True
input_nan = True

save_model = False
model_name = "XGBReg"
parameters = "lagfeatures_removetrend_betterfeatures"

In [37]:
import pandas as pd
from src.models import build_model
from src.processing import process_features,process_target, impute_na
from src.evaluation import evaluate, evaluate_forecaster

# Inport SVR model
from sklearn.svm import SVR
import darts

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
x_train = pd.read_csv("data/x_train.csv", index_col= 'DELIVERY_START', parse_dates=True)
y_train = pd.read_csv("data/y_train.csv", index_col= 'DELIVERY_START', parse_dates=True)
x_test = pd.read_csv("data/x_test.csv", index_col= 'DELIVERY_START', parse_dates=True)

## Evaluation

In [41]:
# Split data
for split_ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    N_train = int(len(x_train)*split_ratio)
    x_train_eval, x_test_eval, train_indices, test_indices = process_features(x_train[:N_train], x_train[N_train:], remove_trend=remove_trend, lag_features=lag_features,impute_nan=input_nan)
    y_train_eval, y_test_eval = y_train[:N_train], y_train[N_train:]
    y_train_eval, y_indices = process_target(y_train_eval, binarize=binarize, impute_nan=input_nan)
    y_test_eval, y_indices = process_target(y_test_eval, binarize=False, impute_nan=input_nan)


    # Train model
    model = build_model(type=model_type)
    if model_type == "forecaster":
        y = darts.timeseries.TimeSeries.from_series(y_train_eval)
        X = darts.timeseries.TimeSeries.from_series(x_train_eval)
        model.fit(y, future_covariates=X)
    else:
        model.fit(x_train_eval, y_train_eval)

    # Evaluate model
    if model_type == "forecaster":
        print(evaluate_forecaster(model, x_test_eval, y_test_eval, train_indices))
    else:
        print(f"Score for split_ratio {split_ratio} : ", evaluate(model, x_test_eval, y_test_eval, test_indices))
    
    print(model.predict(x_test_eval))

# Visualize predictions
import matplotlib.pyplot as plt
if model_type == "forecaster":
    #y_train_pred = model.predict(N_train,future_covariates=X)
    y_test_pred = model.predict(len(x_test_eval),future_covariates=darts.timeseries.TimeSeries.from_series(x_test_eval))
    y_test_pred = y_test_pred.pd_dataframe().squeeze()
else:
    y_train_pred = model.predict(x_train_eval)
    y_test_pred = model.predict(x_test_eval)

# Plot predictions
plt.plot(y_train.index, y_train, label='True')
if model_type != "forecaster":
    plt.plot(x_train_eval.index, y_train_pred, label='Predicted')
plt.plot(x_test_eval.index, y_test_pred, label='Test')
plt.title('Train predictions')
plt.legend()
plt.show()


/mnt/windows/Users/tmdu1/Documents/Thomas/ENS/M2/Cours/Echantillonnage/data_challenge_elmy/src/processing.py:153: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  
/mnt/windows/Users/tmdu1/Documents/Thomas/ENS/M2/Cours/Echantillonnage/data_challenge_elmy/src/processing.py:153: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  
/mnt/windows/Users/tmdu1/Documents/Thomas/ENS/M2/Cours/Echantillonnage/data_challenge_elmy/src/processing.py:153: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  
/mnt/windows/Users/tmdu1/Documents/Thomas/ENS/M2/Cours/Echantillonnage/data_challenge_elmy/src/processing.py:153: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  


                           spot_id_delta
DELIVERY_START                          
2022-01-01 01:00:00+00:00              0
2022-01-01 02:00:00+00:00              0
2022-01-01 03:00:00+00:00              0
2022-01-01 04:00:00+00:00              1
2022-01-01 05:00:00+00:00              1
...                                  ...
2022-02-15 15:00:00+00:00              1
2022-02-15 16:00:00+00:00              1
2022-02-15 17:00:00+00:00              1
2022-02-15 18:00:00+00:00              0
2022-02-15 19:00:00+00:00              0

[1099 rows x 1 columns]
(9770,)
(9545, 1)
[   0    1    2 ... 9767 9768 9769]


IndexError: positional indexers are out-of-bounds

## Prepare submission

In [37]:
proc_x_train, proc_x_test, _,_  = process_features(x_train,x_test,remove_trend=remove_trend, lag_features=lag_features,impute_nan=input_nan)
proc_y_train, y_indices= process_target(y_train, binarize=binarize)

# Fit with full data
model = build_model(type=model_type)
model.fit(proc_x_train, proc_y_train);

In [ ]:
if save_model:
    from datetime import datetime
    y_test_pred = model.predict(proc_x_test)
    print(len(y_test_pred))
    y_test_pred = y_test_pred[x_test_indices[0]]
    print(len(y_test_pred))
    y_test_pred = pd.DataFrame(y_test_pred, index=x_test_raw.index, columns=['spot_id_delta'])
    y_test_pred.to_csv(f'submission/{datetime.today().date()}_{model_name}_{parameters}.csv', index=True)

4944
4942
